<a href="https://colab.research.google.com/github/Volze/PSC/blob/master/kaggle_comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train (1).csv


In [ ]:
import numpy as np
import pandas as pd

data = pd.read_csv('train.csv').to_numpy()

In [ ]:

import re

# Removing latex in text, trailing spaces and text in brackets
def sanitize_text(abstract):
  withoutBracketContent = re.sub(r'\([^)]*\)', '', abstract)
  withoutEndLine = re.sub('\n', ' ', withoutBracketContent.strip())
  return re.sub(r'\$(.*?)\$', '', withoutEndLine.lower())

parsedAbstracts = np.array([sanitize_text(abstract) for abstract in data.T[1]])

trainset = np.array(list(zip(data.T[0], parsedAbstracts, data.T[2])))
labels = np.unique(trainset.T[2])

words = ' '.join(parsedAbstracts)
vocab = np.unique(words.split(' '))

vocab_dict = dict(zip(list(vocab), [i for i in range(len(vocab))]))
# We should remove stop words also

In [ ]:

testset = [None]*len(labels)

for i, label in enumerate(labels):
  testset[i] = [entry[1] for entry in trainset if entry[2] == label]

priors = list(map(lambda x: len(x)/len(parsedAbstracts), testset))

def words_frequencies_for_topic(topic_set):
  words_for_topic = list((' '.join(topic_set)).split(' '))
  return [words_for_topic.count(word) for word in vocab]

freqs = [words_frequencies_for_topic(topic) for topic in testset]

In [ ]:
#Priors are the same, so we their multiplication, so we can omit them in the optimization
count_for_topic = [np.sum(freqs[i]) for i in range(len(labels))]

In [ ]:
import math

def calculate_proba_for_word(wordIndex, topicIndex):
  num = freqs[topicIndex][wordIndex] + 1
  denom = count_for_topic[topicIndex] + len(vocab)
  return math.log(num/denom)

def calculate_posterior(words, topicIndex):
  return np.sum([calculate_proba_for_word(vocab_dict[word], topicIndex) for word in words])

def sanitize_text_text_and_remove_out_of_vocab(abstract):
  sanitized = sanitize_text(abstract)
  words = sanitized.split(' ')
  return [word for word in words if len(np.where(vocab == word)[0]) > 0]

def predict(abstract):
  return labels[np.argmax([calculate_posterior(sanitize_text_text_and_remove_out_of_vocab(abstract), i) for i in range(len(labels))])]

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv


In [ ]:
test = pd.read_csv('test.csv').to_numpy()

In [ ]:
from google.colab import files

results = [predict(x) for x in test.T[1:2]]
pd.DataFrame(results).to_csv("results.csv")
files.download('results.csv') 

In [ ]:
results

['hep-ph',
 'math.CO',
 'math.CO',
 'gr-qc',
 'gr-qc',
 'gr-qc',
 'gr-qc',
 'math.AP',
 'hep-ph']